In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv(r"C:\Users\sairam\Downloads\eeg dataset\50%C&T_PAC&PLV_3conditions.csv")
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F472,F473,F474,F475,F476,F477,F478,F479,F480,Class
0,0.760200,0.760200,0.620000,1.000000,1.000000,0.7602,1.000000,1.000000,0.619900,1.000000,...,0.987219,0.989565,0.998640,1.000000,0.962797,0.979796,0.994741,0.985073,0.987748,C
1,0.760219,1.000000,0.760203,1.000000,1.000000,1.0000,1.000000,1.000000,0.760235,1.000000,...,0.972903,1.000000,0.984484,0.994453,0.992057,0.992039,0.998955,0.993867,0.971642,C
2,0.767665,0.760478,1.000000,1.000000,1.000000,1.0000,0.760250,0.760883,0.760278,0.760712,...,0.993844,0.991813,0.993898,0.990465,0.982123,0.984472,0.980884,0.992068,0.996257,C
3,1.000000,0.760669,1.000000,1.000000,0.760226,1.0000,0.760792,1.000000,0.620244,0.770195,...,0.994312,0.984167,0.997980,0.989754,0.973449,0.995377,0.990504,0.989309,1.000000,C
4,1.000000,0.760199,0.760350,0.760908,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,...,0.980532,0.993955,0.956835,0.985188,1.000000,0.999691,0.976811,0.980823,0.979616,C


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Columns: 481 entries, F1 to Class
dtypes: float64(477), int64(3), object(1)
memory usage: 158.0+ KB


In [3]:
data.isnull().sum()

F1       0
F2       0
F3       0
F4       0
F5       0
        ..
F477     0
F478     0
F479     0
F480     0
Class    0
Length: 481, dtype: int64

In [4]:
data['Class'].value_counts()

T1    12
T2    12
T3    12
C      6
Name: Class, dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
data['Class']=enc.fit_transform(data['Class'])
data['Class'].value_counts()

1    12
2    12
3    12
0     6
Name: Class, dtype: int64

In [6]:
cls_0=data[data['Class']==0]
cls_1=data[data['Class']==1]
cls_2=data[data['Class']==2]
cls_3=data[data['Class']==3]

In [9]:
cls_0=cls_0.sample(250,replace=True)
cls_1=cls_1.sample(250,replace=True)
cls_2=cls_2.sample(250,replace=True)
cls_3=cls_3.sample(250,replace=True)
data = pd.concat([cls_0,cls_1,cls_2,cls_3 ], axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 5 to 14
Columns: 481 entries, F1 to Class
dtypes: float64(477), int32(1), int64(3)
memory usage: 3.7 MB


In [10]:
corr=data.corr()

In [13]:
cor_target = abs(corr["Class"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.7]
relevant_features
#if needed use these features alone. These are highly correlated (only if u dont need to use all 450 columns)

F4       0.700789
F20      0.700298
F36      0.700654
F322     0.728822
F327     0.719131
F338     0.728828
F343     0.718987
F354     0.728792
F359     0.718995
F439     0.784400
Class    1.000000
Name: Class, dtype: float64

In [17]:
y=data['Class']
X=data.drop(['Class'],axis=1)


In [18]:
from sklearn.model_selection import train_test_split

X_train,X_test ,y_train, y_test = train_test_split(X,y,
                                                   test_size=0.30,
                                                   random_state=0,
                                                  shuffle = True,
                                                  stratify = y)

print('training data shape is :{}.'.format(X_train.shape))
print('training label shape is :{}.'.format(y_train.shape))
print('testing data shape is :{}.'.format(X_test.shape))
print('testing label shape is :{}.'.format(y_test.shape))

training data shape is :(700, 480).
training label shape is :(700,).
testing data shape is :(300, 480).
testing label shape is :(300,).


In [22]:
from sklearn.svm import SVC
svc_model = SVC(kernel='rbf',gamma=8)
svc_model.fit(X_train, y_train)

SVC(gamma=8)

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix
predictions= svc_model .predict(X_train)
percentage=svc_model.score(X_train,y_train)
res=confusion_matrix(y_train,predictions)
print("Training confusion matrix")
print(res)
predictions= svc_model .predict(X_test)
train_percentage=svc_model.score(X_train,y_train)
test_percentage=svc_model.score(X_test,y_test)
res=confusion_matrix(y_test,predictions)
print("Testing confusion matrix")
print(res)
# check the accuracy on the training set
print(svc_model.score(X_train, y_train))
print(svc_model.score(X_test, y_test))
print(f"Train set:{len(X_train)}")
print(f"Train Accuracy={train_percentage*100}%")
print(f"Test set:{len(X_test)}")
print(f"Test Accuracy={test_percentage*100}%")

Training confusion matrix
[[175   0   0   0]
 [  0 175   0   0]
 [  0   0 175   0]
 [  0   0   0 175]]
Testing confusion matrix
[[75  0  0  0]
 [ 0 75  0  0]
 [ 0  0 75  0]
 [ 0  0  0 75]]
1.0
1.0
Train set:700
Train Accuracy=100.0%
Test set:300
Test Accuracy=100.0%
